**Deviding dataset into imbalanced clients**

In [ ]:
import os

base_path = r"C:\Users\karin\Documents\Federated_learning\cleaned\Training"
print(os.listdir(base_path))

['glioma', 'meningioma', 'notumor', 'pituitary']


In [ ]:
import os
import random
import shutil
import math

# Base preprocessed dataset
base_path = r"C:\Users\karin\Documents\Federated_learning\cleaned\Training"
client_base = r"C:\Users\karin\Documents\Federated_learning\clients"

# Clients / hospitals
clients = ["client1", "client2", "client3", "client4"]

# Remove old client folders if they exist
if os.path.exists(client_base):
    shutil.rmtree(client_base)
os.makedirs(client_base)

# Class names
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Read all images per class
all_images = {}
for cls in classes:
    cls_path = os.path.join(base_path, cls)
    images = [os.path.join(cls_path, f) for f in os.listdir(cls_path)]
    all_images[cls] = images

# Total images per client (simulate hospital sizes)
total_images_per_client = {
    "client1": 1000,
    "client2": 400,
    "client3": 200,
    "client4": 50
}

# Fully imbalanced class fractions per client (sum not 1, intentional)
class_fractions = {
    "client1": {"glioma": 0.7, "meningioma": 0.2, "pituitary": 0.08, "notumor": 0.02},
    "client2": {"glioma": 0.05, "meningioma": 0.1, "pituitary": 0.7, "notumor": 0.15},
    "client3": {"glioma": 0.1, "meningioma": 0.05, "pituitary": 0.1, "notumor": 0.75},
    "client4": {"glioma": 0.4, "meningioma": 0.3, "pituitary": 0.2, "notumor": 0.1},
}

# Create client datasets
for client in clients:
    client_path = os.path.join(client_base, client, "Training")
    os.makedirs(client_path, exist_ok=True)
    total_imgs = total_images_per_client[client]

    for cls, frac in class_fractions[client].items():
        cls_path = os.path.join(client_path, cls)
        os.makedirs(cls_path, exist_ok=True)

        # Number of images for this class
        num_images = max(1, math.floor(total_imgs * frac))
        # Randomly select images without removing from original
        selected = random.sample(all_images[cls], min(num_images, len(all_images[cls])))

        for img_path in selected:
            shutil.copy(img_path, cls_path)

print("Client datasets created: all clients have all classes, fully imbalanced.")


Client datasets created: all clients have all classes, fully imbalanced.


**Data Augmentation**

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Base path where client datasets are stored
client_base = r"C:\Users\karin\Documents\Federated_learning\clients"

# Define augmentation transforms
train_transform = transforms.Compose([
    transforms.RandomRotation(20),                     # rotation range 20 degrees
    transforms.RandomAffine(
        degrees=0,
        translate=(0.2, 0.2),                         # width and height shift 20%
        shear=20,                                     # shear 20 degrees
        fill=0                                        # fill empty pixels with black
    ),
    transforms.RandomResizedCrop(256, scale=(0.8, 1.2)),  # zoom range ~20%
    transforms.RandomHorizontalFlip(p=0.5),           # horizontal flip
    transforms.ToTensor()
])

# For validation/testing (no augmentation, only resize and tensor)
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Create DataLoaders for each client
client_loaders = {}
batch_size = 16

for client in os.listdir(client_base):
    client_path = os.path.join(client_base, client, "Training")
    if not os.path.isdir(client_path):
        continue

    dataset = datasets.ImageFolder(client_path, transform=train_transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    client_loaders[client] = loader

print("DataLoaders with augmentation created for clients:", list(client_loaders.keys()))


**Feature Extraction**

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained ResNet (without final classification layer)
resnet = models.resnet18(pretrained=True)
modules = list(resnet.children())[:-1]  # remove final FC layer
feature_extractor = nn.Sequential(*modules).to(device)
feature_extractor.eval()  # set to eval mode

# Example: extract features for a batch
for client, loader in client_loaders.items():
    for images, labels in loader:
        images = images.to(device)
        with torch.no_grad():
            features = feature_extractor(images)  # shape: [batch_size, 512, 1, 1]
            features = features.view(features.size(0), -1)  # flatten to [batch_size, 512]


**Feature Selection**

In [ ]:
client_features_dict = {}
client_features_dict = {}

for client, loader in client_loaders.items():
    all_features = []
    all_labels = []
    
    for images, labels in loader:
        images = images.to(device)
        with torch.no_grad():
            features = feature_extractor(images)           # [batch_size, 512, 1, 1]
            features = features.view(features.size(0), -1) # flatten to [batch_size, 512]
        
        all_features.append(features.cpu())  # move to CPU and store
        all_labels.append(labels)            # labels are already CPU tensors

    # Stack all batches to one tensor per client
    client_features_dict[client] = {
        "features": torch.cat(all_features, dim=0),  # [total_images, 512]
        "labels": torch.cat(all_labels, dim=0)       # [total_images]
    }

print("Feature extraction complete. Example shapes:")
for client, data in client_features_dict.items():
    print(client, data["features"].shape, data["labels"].shape)

for client, loader in client_loaders.items():
    all_features = []
    all_labels = []
    
    for images, labels in loader:
        images = images.to(device)
        with torch.no_grad():
            features = feature_extractor(images)           # [batch_size, 512, 1, 1]
            features = features.view(features.size(0), -1) # flatten to [batch_size, 512]
        
        all_features.append(features.cpu())  # move to CPU and store
        all_labels.append(labels)            # labels are already CPU tensors

    # Stack all batches to one tensor per client
    client_features_dict[client] = {
        "features": torch.cat(all_features, dim=0),  # [total_images, 512]
        "labels": torch.cat(all_labels, dim=0)       # [total_images]
    }

print("Feature extraction complete. Example shapes:")
for client, data in client_features_dict.items():
    print(client, data["features"].shape, data["labels"].shape)


**Feature Selection and Federated Learning**

In [ ]:
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA

# -----------------------------
# Step 1: Determine fixed PCA dimension
# -----------------------------
min_samples = min([client_features_dict[c]['features'].shape[0] for c in client_features_dict])
fixed_n_components = min(50, min_samples)  # adjust 50 or lower if some client has very few images
print("Fixed PCA components:", fixed_n_components)

# -----------------------------
# Step 2: Apply PCA per client
# -----------------------------
client_reduced_features = {}
for client in client_features_dict:
    X = client_features_dict[client]['features'].numpy()
    y = client_features_dict[client]['labels'].numpy()
    
    pca = PCA(n_components=fixed_n_components)
    X_reduced = pca.fit_transform(X)
    
    client_reduced_features[client] = {
        "features": X_reduced,
        "labels": y
    }
    
    print(client, "reduced features shape:", X_reduced.shape)

# -----------------------------
# Step 3: Define MLP
# -----------------------------
class MLP(nn.Module):
    def __init__(self, input_dim=fixed_n_components, num_classes=4):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, num_classes)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# -----------------------------
# Step 4: FedAvg function
# -----------------------------
def fed_avg(weights_list):
    avg_weights = {}
    for key in weights_list[0].keys():
        avg_weights[key] = sum([w[key] for w in weights_list]) / len(weights_list)
    return avg_weights

# -----------------------------
# Step 5: Initialize global model
# -----------------------------
global_model = MLP(input_dim=fixed_n_components, num_classes=4)
global_weights = global_model.state_dict()

# -----------------------------
# Step 6: Federated learning loop
# -----------------------------
num_rounds = 5
local_epochs = 2
learning_rate = 0.01

for round in range(num_rounds):
    local_weights = []
    print(f"--- Round {round+1} ---")
    
    for client in client_reduced_features:
        X = torch.tensor(client_reduced_features[client]['features'], dtype=torch.float32)
        y = torch.tensor(client_reduced_features[client]['labels'], dtype=torch.long)
        
        # Initialize local model with global weights
        model = MLP(input_dim=fixed_n_components, num_classes=4)
        model.load_state_dict(copy.deepcopy(global_weights))
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()
        
        # Local training
        for epoch in range(local_epochs):
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
        
        local_weights.append(copy.deepcopy(model.state_dict()))
    
    # FedAvg aggregation
    global_weights = fed_avg(local_weights)
    global_model.load_state_dict(global_weights)

print("Federated training complete. Global model ready for evaluation.")


**Model Evaluation**

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# -----------------------------
# Step 1: Evaluate local models (trained individually per client)
# -----------------------------
print("=== Local Model Evaluation ===")
local_model_performance = {}

for client in client_reduced_features:
    # Initialize a new MLP for this client
    model = MLP(input_dim=fixed_n_components, num_classes=4).to(device)
    
    # Train locally (same as in federated loop)
    X = torch.tensor(client_reduced_features[client]['features'], dtype=torch.float32).to(device)
    y = torch.tensor(client_reduced_features[client]['labels'], dtype=torch.long).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    
    model.train()
    for epoch in range(5):  # local training epochs
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    
    # Evaluate
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        _, preds = torch.max(outputs, 1)
        loss_val = criterion(outputs, y).item()
        acc_val = (preds == y).float().mean().item()
    
    local_model_performance[client] = {"loss": loss_val, "accuracy": acc_val}
    print(f"{client} - Loss: {loss_val:.4f}, Accuracy: {acc_val:.4f}")

# -----------------------------
# Step 2: Evaluate global federated model
# -----------------------------
print("\n=== Global Federated Model Evaluation ===")
global_model.eval()

for client in client_reduced_features:
    X = torch.tensor(client_reduced_features[client]['features'], dtype=torch.float32).to(device)
    y = torch.tensor(client_reduced_features[client]['labels'], dtype=torch.long).to(device)
    
    with torch.no_grad():
        outputs = global_model(X)
        _, preds = torch.max(outputs, 1)
        loss_val = criterion(outputs, y).item()
        acc_val = (preds == y).float().mean().item()
    
    print(f"{client} - Loss: {loss_val:.4f}, Accuracy: {acc_val:.4f}")

# -----------------------------
# Step 3: Global evaluation across all clients combined
# -----------------------------
X_all = np.vstack([client_reduced_features[c]['features'] for c in client_reduced_features])
y_all = np.hstack([client_reduced_features[c]['labels'] for c in client_reduced_features])

X_all_t = torch.tensor(X_all, dtype=torch.float32).to(device)
y_all_t = torch.tensor(y_all, dtype=torch.long).to(device)

with torch.no_grad():
    outputs = global_model(X_all_t)
    _, preds = torch.max(outputs, 1)
    loss_all = criterion(outputs, y_all_t).item()
    acc_all = (preds == y_all_t).float().mean().item()

print(f"\nGlobal Evaluation (all clients combined) - Loss: {loss_all:.4f}, Accuracy: {acc_all:.4f}")
print("\nClassification Report:")
from sklearn.metrics import classification_report
print(classification_report(y_all_t.cpu().numpy(), preds.cpu().numpy(),
                            target_names=['glioma','meningioma','notumor','pituitary']))
